# ติดตั้งไลบรารี

In [ ]:
!pip install --upgrade pip
!pip install openpyxl

In [54]:
# Cell 1: ติดตั้งไลบรารี
import sys
!{sys.executable} -m pip install selenium webdriver_manager openpyxl
print("ติดตั้งไลบรารีเสร็จสิ้น (หากติดตั้งไปแล้ว จะแสดงว่า Requirement already satisfied)")


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### ตรวจหาที่ จะติดตั้ง 

In [55]:
import sys
print(sys.executable)

c:\Users\const\AppData\Local\Programs\Python\Python312\python.exe


In [56]:
# --- Cell 1: Imports และการตั้งค่า ---
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager


import time
import os
from dotenv import load_dotenv

# โหลดค่าจากไฟล์ .env
load_dotenv()

# --- ค่าคงที่ ---
LOGIN_URL = "https://cenproject.rid.go.th/"
YOUR_USERNAME = os.getenv("CEN_USERNAME")
YOUR_PASSWORD = os.getenv("CEN_PASSWORD")
EXCEL_FILE_PATH = 'table_main_2068.xlsx' # <<<<---- ระบุชื่อไฟล์ Excel ของคุณที่นี่
OUTPUT_EXCEL_PATH = 'result_data.xlsx' # <<<<---- ไฟล์ผลลัพธ์ที่จะถูกสร้างขึ้น

print("การตั้งค่าเสร็จสิ้น")

การตั้งค่าเสร็จสิ้น


# 1.เปิด Chrome เพื่อ Login

In [218]:
print(YOUR_USERNAME, YOUR_PASSWORD)

rid2ez2 wa792d


In [219]:
# --- Cell 2: เริ่มต้น WebDriver และ Login ---
print("กำลังเริ่มต้น WebDriver...")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.maximize_window()

print(f"กำลังเข้าสู่หน้า Login: {LOGIN_URL}")
driver.get(LOGIN_URL)

wait = WebDriverWait(driver, 5) # เพิ่มเวลารอเป็น 20 วินาทีเผื่อเว็บโหลดช้า

try:
    # 1. Login
    print("กำลังรอหน้า Login...")
    username_field = wait.until(EC.presence_of_element_located((By.NAME, "Username")))
    password_field = wait.until(EC.presence_of_element_located((By.NAME, "Password")))
    login_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn.btn-primary[type='submit']")))

    print("กำลังกรอกข้อมูล Login...")
    username_field.send_keys(YOUR_USERNAME)
    password_field.send_keys(YOUR_PASSWORD)
    login_button.click()
    print("Login สำเร็จ")

    # 2. กดเข้าระบบติดตาม
    print("กำลังรอลิงก์ 'ระบบติดตาม'...")
    system_tracking_link = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[./img[@alt='banner-track']]")))
    system_tracking_link.click()
    print("คลิก 'ระบบติดตาม' สำเร็จ")

    # --- ส่วนที่แก้ไข ---
    # 3. รอให้หน้ากาก Loading หายไปก่อน
    print("กำลังรอให้หน้า Dashboard โหลดเสร็จ (รอให้ Loading Overlay หายไป)...")
    loading_overlay_xpath = "//div[contains(@style, 'display: block') and contains(@style, 'z-index: 9999')]"
    # เราจะรอจนกว่า Element นี้จะ "มองไม่เห็น" (invisible)
    wait.until(EC.invisibility_of_element_located((By.XPATH, loading_overlay_xpath)))
    print("Loading Overlay หายไปแล้ว หน้าเว็บพร้อมใช้งาน")
except Exception as e:
    print(f"เกิดข้อผิดพลาดระหว่างการ Login หรือการนำทางเริ่มต้น: {e}")
    # driver.quit()

กำลังเริ่มต้น WebDriver...
กำลังเข้าสู่หน้า Login: https://cenproject.rid.go.th/
กำลังรอหน้า Login...
กำลังกรอกข้อมูล Login...
Login สำเร็จ
กำลังรอลิงก์ 'ระบบติดตาม'...
คลิก 'ระบบติดตาม' สำเร็จ
กำลังรอให้หน้า Dashboard โหลดเสร็จ (รอให้ Loading Overlay หายไป)...
Loading Overlay หายไปแล้ว หน้าเว็บพร้อมใช้งาน


# 2.งบประมาณรายจ่ายประจำปี (พ.ร.บ.) 2568

In [185]:
# --- ใส่โค้ดนี้แทนที่ try...except บล็อกเดิมของคุณ ---
def go_to_main_budget_page(driver):
    try:
        # 4. คลิกเมนูหลัก "งบประมาณ"
        # print("กำลังรอเมนู 'งบประมาณ'...")
        # budget_menu_left_nav = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[contains(., 'งบประมาณ')]")))
        # budget_menu_left_nav.click()
        # print("คลิกเมนู 'งบประมาณ' สำเร็จ")

        # 4.1. รอให้หน้า "งบประมาณ" หลักโหลดเสร็จ
        base_budget_url = "https://cenproject.rid.go.th/track/budget?BudgetYear=2025"
        # wait.until(EC.url_to_be(base_budget_url))
        print(f"มาถึงหน้า 'งบประมาณ' หลักแล้ว: {driver.current_url}")

        # 4.2. สร้าง URL เป้าหมายและใช้ driver.get()
        target_url = base_budget_url + "&BudgetSourceID=1"
        # print(f"กำลังนำทางไปยัง URL เป้าหมาย: {target_url}")
        driver.get(target_url)
        
        # 5. รอให้ตารางข้อมูลของหน้าเป้าหมายโหลดเสร็จ (ปรับปรุงจุดนี้)
        print("กำลังรอให้ตารางข้อมูลโครงการโหลด...")
        
        # ลองเพิ่มเวลาในการรอให้มากขึ้น เผื่อข้อมูลเยอะแล้วโหลดช้า
        # long_wait = WebDriverWait(driver, 5) # สร้าง wait ใหม่ที่รอนานขึ้นเป็น 30 วินาที
        
        # เราจะรอ "Container" ของตารางแทน ID ที่อาจจะเปลี่ยนไป
        # ลองหา class ของ <div> ที่ครอบตารางทั้งหมด เช่น 'card-body' หรือ 'table-responsive'
        # **** ปรับแก้ 'card-body' เป็น class ที่ถูกต้องจากหน้าเว็บ ****
        # table_container = long_wait.until(EC.presence_of_element_located((By.CLASS_NAME, "card-body")))
        
        # หลังจากเจอ Container แล้ว ค่อยเช็คว่ามีตารางอยู่ข้างในหรือไม่
        # table_element = table_container.find_element(By.TAG_NAME, "table")
        
        print(f"หน้าตางบประมาณรายจ่ายประจำปี (พ.ร.บ.) โหลดเรียบร้อยแล้ว (พบตาราง ID: {table_element.get_attribute('id')})")

    except Exception as e:
        # แก้ไขข้อความ Error ตามที่ต้องการ
        print("\n--- การนำทางสำเร็จ แต่เกิดข้อผิดพลาดในการรอ Element ---")
        print(f"ข้อผิดพลาด: ไม่สามารถหา Element ที่ต้องการได้ภายในเวลาที่กำหนด")
        print(f"URL ปัจจุบันที่เกิดปัญหา: {driver.current_url}")
        print("สาเหตุที่เป็นไปได้: ID หรือ Class ของตารางข้อมูลในหน้านี้อาจไม่ตรงกับที่ระบุในโค้ด")


In [220]:
go_to_main_budget_page(driver)

มาถึงหน้า 'งบประมาณ' หลักแล้ว: https://cenproject.rid.go.th/track/project?BudgetYear=2025
กำลังรอให้ตารางข้อมูลโครงการโหลด...

--- การนำทางสำเร็จ แต่เกิดข้อผิดพลาดในการรอ Element ---
ข้อผิดพลาด: ไม่สามารถหา Element ที่ต้องการได้ภายในเวลาที่กำหนด
URL ปัจจุบันที่เกิดปัญหา: https://cenproject.rid.go.th/track/budget?BudgetYear=2025&BudgetSourceID=1
สาเหตุที่เป็นไปได้: ID หรือ Class ของตารางข้อมูลในหน้านี้อาจไม่ตรงกับที่ระบุในโค้ด


# ดึงค่ารายการและข้อมูลการเบิกจ่าย จาก Excel

## test วิ่งเข้า งานรักษาบริเวณ สชป.2

In [188]:
# --- Cell 4: เข้าหน้ารายละเอียดของงานรักษาบริเวณ สชป.2 ---
print(f"กำลังค้นหาแถวของ: '{"งานรักษาบริเวณ สชป.2"}'...")
project_row_xpath = f"//tbody/tr[td[2][normalize-space()='{"งานรักษาบริเวณ สชป.2"}']]"
project_row = wait.until(EC.visibility_of_element_located((By.XPATH, project_row_xpath)))
action_button_xpath = ".//div[@data-tip='เรียกดูข้อมูลแถบเดิม']/button"
action_button = project_row.find_element(By.XPATH, action_button_xpath)
print("พบปุ่ม Action, กำลังคลิกเพื่อเข้าหน้ารายละเอียด...")
action_button.click()
# Step 2: รอให้หน้ารายละเอียด (ตารางงานย่อย) โหลด
wait.until(EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'table-zebra')]//tbody")))
print("เข้าสู่หน้ารายละเอียดงานย่อยสำเร็จ.")

กำลังค้นหาแถวของ: 'งานรักษาบริเวณ สชป.2'...
พบปุ่ม Action, กำลังคลิกเพื่อเข้าหน้ารายละเอียด...
เข้าสู่หน้ารายละเอียดงานย่อยสำเร็จ.


In [196]:
go_to_main_budget_page(driver)

มาถึงหน้า 'งบประมาณ' หลักแล้ว: https://cenproject.rid.go.th/track/budget/111?BudgetYear=2025&BudgetSourceID=1&OrganizationName=%E0%B9%82%E0%B8%84%E0%B8%A3%E0%B8%87%E0%B8%81%E0%B8%B2%E0%B8%A3%E0%B8%8A%E0%B8%A5%E0%B8%9B%E0%B8%A3%E0%B8%B0%E0%B8%97%E0%B8%B2%E0%B8%99%E0%B9%80%E0%B8%8A%E0%B8%B5%E0%B8%A2%E0%B8%87%E0%B8%A3%E0%B8%B2%E0%B8%A2
กำลังรอให้ตารางข้อมูลโครงการโหลด...

--- การนำทางสำเร็จ แต่เกิดข้อผิดพลาดในการรอ Element ---
ข้อผิดพลาด: ไม่สามารถหา Element ที่ต้องการได้ภายในเวลาที่กำหนด
URL ปัจจุบันที่เกิดปัญหา: https://cenproject.rid.go.th/track/budget?BudgetYear=2025&BudgetSourceID=1
สาเหตุที่เป็นไปได้: ID หรือ Class ของตารางข้อมูลในหน้านี้อาจไม่ตรงกับที่ระบุในโค้ด


In [195]:
# --- Cell 4: เข้าหน้ารายละเอียดของงานรักษาบริเวณ สชป.2 ---
print(f"กำลังค้นหาแถวของ: '{"โครงการชลประทานเชียงราย"}'...")
project_row_xpath = f"//tbody/tr[td[2][normalize-space()='{"โครงการชลประทานเชียงราย"}']]"
project_row = wait.until(EC.visibility_of_element_located((By.XPATH, project_row_xpath)))
action_button_xpath = ".//div[@data-tip='เรียกดูข้อมูลแถบเดิม']/button"
action_button = project_row.find_element(By.XPATH, action_button_xpath)
print("พบปุ่ม Action, กำลังคลิกเพื่อเข้าหน้ารายละเอียด...")
action_button.click()
# Step 2: รอให้หน้ารายละเอียด (ตารางงานย่อย) โหลด
wait.until(EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'table-zebra')]//tbody")))
print("เข้าสู่หน้ารายละเอียดงานย่อยสำเร็จ.")

กำลังค้นหาแถวของ: 'โครงการชลประทานเชียงราย'...
พบปุ่ม Action, กำลังคลิกเพื่อเข้าหน้ารายละเอียด...
เข้าสู่หน้ารายละเอียดงานย่อยสำเร็จ.


In [169]:
# --- Cell 4: เข้าหน้ารายละเอียดของงานรักษาบริเวณ สชป.2 ---
print(f"กำลังค้นหาแถวของ: '{"โครงการส่งน้ำและบำรุงรักษากิ่วลม-กิ่วคอหมา"}'...")
project_row_xpath = f"//tbody/tr[td[2][normalize-space()='{"โครงการส่งน้ำและบำรุงรักษากิ่วลม-กิ่วคอหมา"}']]"
project_row = wait.until(EC.visibility_of_element_located((By.XPATH, project_row_xpath)))
action_button_xpath = ".//div[@data-tip='เรียกดูข้อมูลแถบเดิม']/button"
action_button = project_row.find_element(By.XPATH, action_button_xpath)
print("พบปุ่ม Action, กำลังคลิกเพื่อเข้าหน้ารายละเอียด...")
action_button.click()
# Step 2: รอให้หน้ารายละเอียด (ตารางงานย่อย) โหลด
wait.until(EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'table-zebra')]//tbody")))
print("เข้าสู่หน้ารายละเอียดงานย่อยสำเร็จ.")

กำลังค้นหาแถวของ: 'โครงการส่งน้ำและบำรุงรักษากิ่วลม-กิ่วคอหมา'...
พบปุ่ม Action, กำลังคลิกเพื่อเข้าหน้ารายละเอียด...
เข้าสู่หน้ารายละเอียดงานย่อยสำเร็จ.


## test วิ่งเข้า งานรักษาบริเวณ สชป.2 เป็น function

In [190]:
# --- [ฟังก์ชันใหม่] สำหรับค้นหาและคลิกเข้าหน้ารายละเอียดขององค์กร ---


def navigate_to_organization_details(driver, wait, org_name):
    print(f"\n--- กำลังพยายามเข้าสู่หน้ารายละเอียดของ: '{org_name}' ---")
    try:
        # Step 1: ค้นหาแถวขององค์กร
        print(f"กำลังค้นหาแถว '{org_name}'...")
        project_row_xpath = f"//tbody/tr[td[2][normalize-space()='{org_name}']]"
        project_row = wait.until(EC.visibility_of_element_located((By.XPATH, project_row_xpath)))
        
        # Step 2: ค้นหาและคลิกปุ่ม Action
        action_button_xpath = ".//div[@data-tip='เรียกดูข้อมูลแถบเดิม']/button"
        action_button = project_row.find_element(By.XPATH, action_button_xpath)
        
        print("พบปุ่ม Action, กำลังใช้ JavaScript เพื่อคลิก...")
        driver.execute_script("arguments[0].click();", action_button)
        
        # Step 3: รอให้หน้ารายละเอียดโหลดเสร็จ
        # รอให้ตารางของหน้ารายละเอียดปรากฏขึ้น
        wait.until(EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'table-zebra')]//tbody")))
        
        print(f"เข้าสู่หน้ารายละเอียดของ '{org_name}' สำเร็จ!")
        
        return True

    except (TimeoutException, NoSuchElementException, ElementClickInterceptedException) as e:
        print(f"!!! เกิดข้อผิดพลาดขณะพยายามเข้าสู่หน้ารายละเอียดของ '{org_name}' !!!")
        print(f"Error: {type(e).__name__} - {str(e).splitlines()[0]}")
        return False
    except Exception as e:
        print(f"!!! เกิดข้อผิดพลาดที่ไม่คาดคิด: {e}")
        return False

### next page 1 kick

In [202]:
# --- [ฟังก์ชันสำหรับคลิกหน้าถัดไป - ฉบับ Reusable] ---
import re
from selenium.webdriver.common.action_chains import ActionChains

def go_to_next_page(driver, wait, table_xpath):
    
    print("\n--- กำลังตรวจสอบ Pagination ---")
    print(table_xpath)
    try:
        # 1. รอให้ตารางของหน้าปัจจุบันพร้อม และ "จำ" ไว้
        
        try:
            current_table = wait.until(EC.presence_of_element_located((By.XPATH, table_xpath)))
        except TimeoutException:
            print(f"!!! ไม่พบตารางตาม XPath ที่ระบุ: {table_xpath} !!!")
            return False

        # 2. ตรวจสอบว่ามีหน้าถัดไปหรือไม่
        # ดึงข้อความ "แสดงข้อมูล..." ซึ่งมักจะอยู่ใกล้ๆ ตาราง
        pagination_text_element = driver.find_element(By.XPATH, "//div[contains(text(), 'แสดงข้อมูล')]")
        pagination_text = pagination_text_element.text
        numbers = re.findall(r'\d+', pagination_text)
        
        can_go_next = False
        if len(numbers) >= 3:
            last_item_shown = int(numbers[1])
            total_items = int(numbers[2])
            if last_item_shown < total_items:
                can_go_next = True
        
        # 3. ตัดสินใจคลิก "Next" ถ้าทำได้
        if can_go_next:
            print(" -> พบว่ามีหน้าถัดไป, กำลังคลิก 'Next'...")
            
            # ใช้ CSS Selector ที่น่าจะใช้ร่วมกันได้
            next_page_button_selector = "span.mx-4 + button"
            next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, next_page_button_selector)))
            
            # ใช้ ActionChains เพื่อคลิกเหมือนมนุษย์
            ActionChains(driver).move_to_element(next_button).click().perform()

            print(" -> เปลี่ยนหน้าสำเร็จ!")
            return True # คืนค่าว่าไปหน้าต่อไปได้
        else:
            print(" -> เป็นหน้าสุดท้ายแล้ว.")
            return False # คืนค่าว่าเป็นหน้าสุดท้าย

    except NoSuchElementException:
        print(" -> ไม่พบข้อความ Pagination, ถือว่าเป็นหน้าสุดท้าย")
        return False
    except Exception as e:
        print(f"!!! เกิดข้อผิดพลาดร้ายแรงขณะพยายามไปหน้าถัดไป: {e} !!!")
        return False

In [204]:
# ---ตาราง พรบ 2568+ตารางรายการย่อย---

# กำหนด XPath ของ tbody ของตารางเป้าหมาย
main_table_xpath = "//table[contains(@class, 'table-zebra')]//tbody"

# ประมวลผลข้อมูลในหน้าแรก
print("--- กำลังประมวลผลข้อมูลตารางหลักหน้าแรก ---")
# ... (โค้ดอ่านข้อมูลของคุณ) ...

# เรียกใช้ฟังก์ชันเพื่อไปหน้าถัดไป
if go_to_next_page(driver, wait, main_table_xpath):
    print("มีหน้าถัดไปสำหรับตารางหลัก, กำลังประมวลผล...")
else:
    print("ไม่มีหน้าถัดไปสำหรับตารางหลัก")

--- กำลังประมวลผลข้อมูลตารางหลักหน้าแรก ---

--- กำลังตรวจสอบ Pagination ---
//table[contains(@class, 'table-zebra')]//tbody
 -> พบว่ามีหน้าถัดไป, กำลังคลิก 'Next'...
 -> เปลี่ยนหน้าสำเร็จ!
มีหน้าถัดไปสำหรับตารางหลัก, กำลังประมวลผล...


In [221]:
# --- [โค้ดหลัก] สำหรับวนลูปและเรียกใช้ฟังก์ชัน ---

# รายชื่อองค์กรที่ต้องการทำงานด้วย
organization_names = [
    "งานรักษาบริเวณ สชป.2",
    "ศูนย์ภูฟ้าพัฒนา",
    "ส่วนจัดสรรน้ำและบำรุงรักษา สชป.2",
    "ส่วนวิศวกรรมบริหาร สชป.2",
    "ส่วนเครื่องจักรกล สชป.2",
    "โครงการก่อสร้างชลประทานที่ 2",
    "โครงการชลประทานน่าน",
    "โครงการชลประทานพะเยา",
    "โครงการชลประทานลำปาง",
    "โครงการชลประทานเชียงราย",
    # "โครงการส่งน้ำและบำรุงรักษากิ่วลม-กิ่วคอหมา",
    # "โครงการส่งน้ำและบำรุงรักษาแม่ลาว",
    # "โครงการส่งน้ำและบำรุงรักษาแม่วัง"
]
TARGET_TABLE_URL = "https://cenproject.rid.go.th/track/budget?BudgetYear=2025&BudgetSourceID=1"
main_table_xpath = "//table[contains(@class, 'table-zebra')]//tbody"

# (สมมติว่า driver และ wait ถูกสร้างไว้แล้ว และอยู่ในหน้าตารางหน่วยงานหลัก)

for name in organization_names:
    
    # เรียกใช้ฟังก์ชันที่เราสร้างขึ้น
    success = navigate_to_organization_details(driver, wait, name)
    
    # ตรวจสอบว่าฟังก์ชันทำงานสำเร็จหรือไม่
    if success:
        # --- ถ้าสำเร็จ ก็เริ่มทำงานในหน้ารายละเอียดได้เลย ---
        print(f"--- เริ่มประมวลผลข้อมูลในหน้ารายละเอียดของ '{name}' ---")
        
        
  













        
        # time.sleep(4)
        driver.get(TARGET_TABLE_URL) # ใช้ get เพื่อความแน่นอน
        wait.until(EC.presence_of_element_located((By.XPATH, "//table//tbody")))
        
    else:
        # ถ้าฟังก์ชันทำงานไม่สำเร็จ (หาไม่เจอ)
        print(f"--- ไม่สามารถเข้าสู่หน้ารายละเอียดของ '{name}' ได้, ข้ามไปรายการถัดไป ---")
        # ถ้าไม่เจอ รายการใน organization_names ให้ go_to_next_page
        if go_to_next_page(driver, wait, main_table_xpath):
            print("มีหน้าถัดไปสำหรับตารางหลัก, กำลังประมวลผล...")

print("\n========== การทำงานกับทุกองค์กรเสร็จสิ้น ==========")


--- กำลังพยายามเข้าสู่หน้ารายละเอียดของ: 'งานรักษาบริเวณ สชป.2' ---
กำลังค้นหาแถว 'งานรักษาบริเวณ สชป.2'...
พบปุ่ม Action, กำลังใช้ JavaScript เพื่อคลิก...
เข้าสู่หน้ารายละเอียดของ 'งานรักษาบริเวณ สชป.2' สำเร็จ!
--- เริ่มประมวลผลข้อมูลในหน้ารายละเอียดของ 'งานรักษาบริเวณ สชป.2' ---

--- กำลังพยายามเข้าสู่หน้ารายละเอียดของ: 'ศูนย์ภูฟ้าพัฒนา' ---
กำลังค้นหาแถว 'ศูนย์ภูฟ้าพัฒนา'...
พบปุ่ม Action, กำลังใช้ JavaScript เพื่อคลิก...
เข้าสู่หน้ารายละเอียดของ 'ศูนย์ภูฟ้าพัฒนา' สำเร็จ!
--- เริ่มประมวลผลข้อมูลในหน้ารายละเอียดของ 'ศูนย์ภูฟ้าพัฒนา' ---

--- กำลังพยายามเข้าสู่หน้ารายละเอียดของ: 'ส่วนจัดสรรน้ำและบำรุงรักษา สชป.2' ---
กำลังค้นหาแถว 'ส่วนจัดสรรน้ำและบำรุงรักษา สชป.2'...
พบปุ่ม Action, กำลังใช้ JavaScript เพื่อคลิก...
เข้าสู่หน้ารายละเอียดของ 'ส่วนจัดสรรน้ำและบำรุงรักษา สชป.2' สำเร็จ!
--- เริ่มประมวลผลข้อมูลในหน้ารายละเอียดของ 'ส่วนจัดสรรน้ำและบำรุงรักษา สชป.2' ---

--- กำลังพยายามเข้าสู่หน้ารายละเอียดของ: 'ส่วนวิศวกรรมบริหาร สชป.2' ---
กำลังค้นหาแถว 'ส่วนวิศวกรรมบริหาร สชป.2'...
พบปุ่ม

### test ดึงรายการ + ร้อยละเบิกจ่าย

In [222]:
df = pd.read_excel("table_main_2568.xlsx")

In [223]:
subtask_rows = driver.find_elements(By.XPATH, "//table[contains(@class, 'table-zebra')]//tbody/tr")
for index, row_elem in enumerate(subtask_rows, start=1):
    subtask_name = row_elem.find_element(By.XPATH, ".//td[2]").text.strip()
    subtask_percen_dis = row_elem.find_element(By.XPATH, ".//td[9]").text.strip()

    # ค้นหาแถวใน DataFrame ที่ตรงกับ subtask_name
    matched = df[df['ชื่อ / ประมาณการ'] == subtask_name]
    if not matched.empty:
        # ดึงชื่อหน่วยงานระดับโครงการ (คอลัมน์ E)
        project_name = matched.iloc[0]['ชื่อหน่วยงานระดับโครงการ']
        
        print(f"งานย่อยที่ {index}:|หน่วยงาน: {project_name} | ชื่องาน {subtask_name} เบิกจ่ายร้อยละ :{subtask_percen_dis} ")
        # เช็ค subtask_percen_dis <= 100 ถ้า เป็นจริง ให้กด /html/body/div[2]/div[1]/main/main/div[4]/div[1]/table/tbody/tr[1]/td[13]/div/div[1]
        
        # if float(subtask_percen_dis) < 100:
        #     print(float(subtask_percen_dis))
        #     # กดปุ่มที่ต้องการ
        #     button_xpath = "/html/body/div[2]/div[1]/main/main/div[4]/div[1]/table/tbody/tr[1]/td[13]/div/div[1]"
        #     button = driver.find_element(By.XPATH, button_xpath)
        #     button.click()
        #     print("กดปุ่มเรียบร้อยแล้ว")
        # กลับไปที่หน้าเดิม
        # รอให้โหดหน้าเว็บเสร็จ
        # time.sleep(10)
           
    
    else:
        project_name = "ไม่พบใน Excel"
    
    # กลับไปที่หน้ารายการย่อย https://cenproject.rid.go.th/track/budget/111?BudgetYear=2025&BudgetSourceID=1&OrganizationName=
    
    



งานย่อยที่ 1:|หน่วยงาน: โครงการก่อสร้างชลประทานที่ 2 | ชื่องาน (197) ฝายขำพร้อมระบบส่งน้ำ ตำบลงิม อำเภอปง จังหวัดพะเยา - [ 10336 ] เบิกจ่ายร้อยละ :89.08 
งานย่อยที่ 2:|หน่วยงาน: โครงการก่อสร้างชลประทานที่ 2 | ชื่องาน (176) ฝายบ่อเป๊กพร้อมระบบส่งน้ำ ตำบลเรือง อำเภอเมืองน่าน จังหวัดน่าน - [ 10315 ] เบิกจ่ายร้อยละ :79.84 
งานย่อยที่ 3:|หน่วยงาน: โครงการก่อสร้างชลประทานที่ 2 | ชื่องาน (175) ฝายห้วยฮ้อมพร้อมระบบส่งน้ำ ตำบลกองควาย อำเภอเมืองน่าน จังหวัดน่าน - [ 10314 ] เบิกจ่ายร้อยละ :80.06 
งานย่อยที่ 4:|หน่วยงาน: โครงการก่อสร้างชลประทานที่ 2 | ชื่องาน (174) ฝายห้วยแอบพร้อมระบบส่งน้ำ ตำบลศรีษะเกษ อำเภอนาน้อย จังหวัดน่าน - [ 10313 ] เบิกจ่ายร้อยละ :86.18 
งานย่อยที่ 5:|หน่วยงาน: โครงการก่อสร้างชลประทานที่ 2 | ชื่องาน (1.94) ซ่อมแซมอาคารบ้านพักข้าราชการระดับบริหาร 2 แห่ง โครงการก่อสร้าง สำนักงานชลประทานที่ 2 ตำบลแม่สรวย อำเภอแม่สรวย จังหวัดเชียงราย - [ 97 ] เบิกจ่ายร้อยละ :93.07 
งานย่อยที่ 6:|หน่วยงาน: โครงการก่อสร้างชลประทานที่ 2 | ชื่องาน (1.93) ซ่อมแซมอาคารบ้านพัก 2 ชั้น แห่งที่ 1 โครงการ

### วน เข้า กรอก รายงานเบิกจ่าย

In [228]:
# --- [โค้ดสำหรับเช็คเงื่อนไข, ค้นหาใน Excel, และคลิกเข้าไปในแต่ละแถว - ฉบับปรับปรุง] ---
# สมมติว่า df และ wait ถูกสร้างไว้แล้ว
def extract_budget_data(driver, wait):
    # ดึงชื่อ งานจาก xpath/html/body/div[2]/div[1]/main/main/div[1]
    xpath_get_name = "//html/body/div[2]/div[1]/main/main/div[1]"
    name_element = wait.until(EC.presence_of_element_located((By.XPATH, xpath_get_name)))
    name = name_element.text.strip()

    print("\n--- กำลังประมวลผลข้อมูลในหน้ารายละเอียดงานย่อย ---")
    print(f"รายการงาน: {name}...")
    try:
        # 1. หาจำนวนแถวทั้งหมดในหน้าปัจจุบัน
        tbody = wait.until(EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'table-zebra')]//tbody")))
        subtask_rows_count = len(tbody.find_elements(By.XPATH, ".//tr"))
        print(f"พบ {subtask_rows_count} งานย่อยในหน่วยงานนี้. กำลังตรวจสอบเงื่อนไข...")

        # สร้าง Flag เพื่อบอกว่าเราคลิกสำเร็จแล้วหรือยัง
        action_completed = False 
        
        # 2. วนลูปด้วย index เพื่อให้สามารถกลับมาทำงานต่อได้หลังการคลิก
        for i in range(subtask_rows_count):
            # ใช้ i+1 เพราะ index ของ Python เริ่มที่ 0 แต่ XPath เริ่มที่ 1
            print(f"\n--- กำลังตรวจสอบแถวที่ {i + 1} ---")
            
            try:
                # 3. สร้าง XPath สำหรับแถวปัจจุบันขึ้นมาใหม่ทุกครั้ง
                # เราต้องค้นหาจากทั้งหน้า เพราะหลังจากคลิกกลับมา DOM อาจจะเปลี่ยนไป
                current_row_xpath = f"(//table[contains(@class, 'table-zebra')]//tbody/tr)[{i + 1}]"
                
                # 4. ดึงข้อมูล %เบิกจ่าย และชื่องานจากแถวปัจจุบัน
                current_row_element = wait.until(EC.visibility_of_element_located((By.XPATH, current_row_xpath)))

                subtask_percen_dis_text = current_row_element.find_element(By.XPATH, ".//td[9]").text.strip()
                subtask_name = current_row_element.find_element(By.XPATH, ".//td[2]").text.strip()
                
                # ข้ามแถวที่ไม่มีข้อมูลเปอร์เซ็นต์
                if not subtask_percen_dis_text:
                    print("  -> ไม่มีข้อมูล %เบิกจ่าย, ข้ามไป")
                    continue
                
                # ค้นหาใน DataFrame
                matched = df[df['ชื่อ / ประมาณการ'] == subtask_name]
                if not matched.empty:
                    project_name = matched.iloc[0]['ชื่อหน่วยงานระดับโครงการ']
                    print(f"  หน่วยงาน: {project_name} | ชื่องาน: {subtask_name[:40]}... | %เบิกจ่าย: {subtask_percen_dis_text}")

                    # 5. แปลงค่าและตรวจสอบเงื่อนไข
                    percentage_value = float(subtask_percen_dis_text)
                    if percentage_value < 100:
                        print(f"  -> [ผ่านเงื่อนไข] %เบิกจ่าย < 100. กำลังคลิก 'เรียกดูข้อมูลแถบเดิม'...")
                        
                        # 6. สร้าง XPath สำหรับปุ่ม "เรียกดูข้อมูลแถบเดิม" ของแถวปัจจุบัน (แบบไดนามิก)
                        # ใช้ XPath ที่อ้างอิงจาก data-tip ซึ่งทนทานกว่า
                        action_button_xpath = f"{current_row_xpath}/td[13]//div[@data-tip='เรียกดูข้อมูลแถบเดิม']/button"
                        
                        action_button = wait.until(EC.element_to_be_clickable((By.XPATH, action_button_xpath)))
                        
                        # ใช้ JavaScript click เพื่อความแน่นอน
                        driver.execute_script("arguments[0].click();", action_button)
                        
                        # --- ที่ตรงนี้ คือจุดที่เราจะทำงานในหน้ารายละเอียดสุดท้าย ---
                        
                        # ตัวอย่าง: รอให้หน้าใหม่โหลดเสร็จ
                        final_page_element_xpath = "//h2[contains(text(), 'งวดการเบิกจ่าย')]"
                        
                        print("  -> คลิกสำเร็จ! กำลังรอหน้า 'การเบิกจ่ายงบประมาณ' โหลด...")
                        # wait.until(EC.visibility_of_element_located((By.XPATH, final_page_element_xpath)))


                        
                        print("  -> คลิกสำเร็จ! เข้าสู่หน้า การเบิกจ่ายงบประมาณ...")
                        action_completed = True
                        break
                        
                    else:
                        print("  -> [ไม่ผ่านเงื่อนไข] %เบิกจ่ายไม่น้อยกว่า 100, ข้ามไป")
                else:
                    print(f"  ชื่องาน: {subtask_name[:40]}... | ไม่พบข้อมูลใน Excel, ข้ามไป")
            

            except (NoSuchElementException, TimeoutException, ValueError) as e:
                print(f"  -> เกิดข้อผิดพลาดในการประมวลผลแถวที่ {i+1}: {e}")
                continue
            



            # driver.back() 
    except Exception as e:
        print(f"เกิดข้อผิดพลาดในการค้นหาตาราง: {e}")

In [229]:
extract_budget_data(driver, wait)


--- กำลังประมวลผลข้อมูลในหน้ารายละเอียดงานย่อย ---
รายการงาน: คัดกรองข้อมูล...
พบ 6 งานย่อยในหน่วยงานนี้. กำลังตรวจสอบเงื่อนไข...

--- กำลังตรวจสอบแถวที่ 1 ---
  หน่วยงาน: โครงการก่อสร้างชลประทานที่ 2 | ชื่องาน: (197) ฝายขำพร้อมระบบส่งน้ำ ตำบลงิม อำเภอ... | %เบิกจ่าย: 89.08
  -> [ผ่านเงื่อนไข] %เบิกจ่าย < 100. กำลังคลิก 'เรียกดูข้อมูลแถบเดิม'...
  -> คลิกสำเร็จ! กำลังรอหน้า 'การเบิกจ่ายงบประมาณ' โหลด...
  -> คลิกสำเร็จ! เข้าสู่หน้า การเบิกจ่ายงบประมาณ...


###  state การเพิ่ม ผลการเบิกจ่ายสะสม โดยเช็คเดือนปัจจุบันก่อน

In [225]:
import datetime
                    
try:
    # --- Step 1: สร้างข้อความของเดือนปัจจุบัน (เหมือนเดิม) ---
    thai_months = [
        "มกราคม", "กุมภาพันธ์", "มีนาคม", "เมษายน", "พฤษภาคม", "มิถุนายน",
        "กรกฎาคม", "สิงหาคม", "กันยายน", "ตุลาคม", "พฤศจิกายน", "ธันวาคม"
    ]
    now = datetime.datetime.now()
    current_month_name = thai_months[now.month - 1]
    current_buddhist_year = now.year + 543
    target_month_text = f"{current_month_name} {current_buddhist_year}"
    print(f"กำลังค้นหาแถวที่มีข้อความ: '{target_month_text}'")

    # --- Step 2: หาแถวทั้งหมดในตาราง ---
    table_body_xpath = "//table[contains(@class, 'table-zebra')]//tbody"
    tbody = wait.until(EC.presence_of_element_located((By.XPATH, table_body_xpath)))
    all_rows = tbody.find_elements(By.XPATH, ".//tr")
    
    if not all_rows:
        print("!!! ไม่พบแถวข้อมูลในตาราง !!!")
        

    # --- Step 3: วนลูปเพื่อหา Index ของแถวที่มีเดือนปัจจุบัน ---
    target_row_index = -1
    for index, row in enumerate(all_rows):
        try:
            # ดึงข้อความจากคอลัมน์แรกของแต่ละแถว
            month_cell_text = row.find_element(By.XPATH, ".//td[1]").text.strip()
            if month_cell_text == target_month_text:
                # ถ้าเจอเดือนที่ตรงกัน ให้เก็บ index ไว้แล้วหยุดลูป
                target_row_index = index 
                print(f"พบเดือนเป้าหมายที่แถว index {target_row_index} (XPath tr[{target_row_index + 1}])")
                break
        except (NoSuchElementException):
            # ข้ามแถวที่อาจมีปัญหา
            continue
    
    # --- Step 4: ตรวจสอบว่าเจอแถวหรือไม่ และสร้าง XPath สุดท้าย ---
    if target_row_index != -1:
        # ถ้าเจอแถว (target_row_index ไม่ใช่ -1)
        # สร้าง XPath แบบไดนามิกและแข็งแกร่งสำหรับเป้าหมาย
        # เราจะใช้ Relative XPath จาก tbody ซึ่งดีกว่า Absolute XPath มาก
        
        # โครงสร้าง: (//tbody/tr)[index + 1]/td[7]//div/div
        target_element_xpath = f"({table_body_xpath}/tr)[{target_row_index + 1}]/td[7]//div/div"
        
        print(f"กำลังค้นหา Element เป้าหมายด้วย XPath: {target_element_xpath}")
        
        # รอให้ Element นั้นปรากฏและพร้อมใช้งาน
        target_element = wait.until(EC.element_to_be_clickable((By.XPATH, target_element_xpath)))
        print(target_element)
        # ใช้ JavaScript click เพื่อความแน่นอน
        driver.execute_script("arguments[0].click();", target_element)
        
    else:
        # ถ้าวนลูปจนจบแล้วยังหาเดือนปัจจุบันไม่เจอ
        print(f"!!! ไม่พบแถวของเดือน '{target_month_text}' ในตาราง !!!")
        

except Exception as e:
    print(f"!!! เกิดข้อผิดพลาดร้ายแรงในฟังก์ชัน: {e}")

กำลังค้นหาแถวที่มีข้อความ: 'กรกฎาคม 2568'
!!! ไม่พบแถวของเดือน 'กรกฎาคม 2568' ในตาราง !!!


In [105]:
# เช็ค subtask_percen_dis <= 100 ถ้า เป็นจริง ให้กด /html/body/div[2]/div[1]/main/main/div[4]/div[1]/table/tbody/tr[1]/td[13]/div/div[1]
if subtask_percen_dis < '100':
    # กดปุ่มที่ต้องการ
    button_xpath = "/html/body/div[2]/div[1]/main/main/div[4]/div[1]/table/tbody/tr[1]/td[13]/div/div[1]"
    button = driver.find_element(By.XPATH, button_xpath)
    button.click()
    print("กดปุ่มเรียบร้อยแล้ว")



### test เข้ารายการย่อย ดึง เบิกจ่ายสะสม จาก excel ที่ชื่อตรงกันมา

### test วิ่ง เข้า รายงานการเบิกจ่าย เช็คจำนวนรายการ

In [119]:
# --- ตัวอย่างการเรียกใช้กับตารางที่สอง ---

# กำหนด XPath ของ tbody ของตารางเป้าหมายที่สอง
other_table_xpath = "/html/body/div[2]/div[1]/main/main/div[2]/div[2]/div/div[2]/div[1]/div/div[4]/div[1]/table/tbody"

# (สมมติว่าคุณได้คลิกเข้ามาจนถึงหน้าที่มีตารางนี้แล้ว)

# ประมวลผลข้อมูลในหน้าแรกของตารางนี้
print("\n--- กำลังประมวลผลข้อมูลตารางที่สองหน้าแรก ---")
# ... (โค้ดอ่านข้อมูลสำหรับตารางนี้) ...

# เรียกใช้ฟังก์ชันเดิมเพื่อไปหน้าถัดไปของตารางนี้
if go_to_next_page(driver, wait, other_table_xpath):
    print("ตอนนี้คุณอยู่ที่หน้า 2 ของตารางที่สองแล้ว")
else:
    print("ไม่มีหน้าถัดไปสำหรับตารางที่สอง")


--- กำลังประมวลผลข้อมูลตารางที่สองหน้าแรก ---

--- กำลังตรวจสอบ Pagination ---
/html/body/div[2]/div[1]/main/main/div[2]/div[2]/div/div[2]/div[1]/div/div[4]/div[1]/table/tbody
!!! ไม่พบตารางตาม XPath ที่ระบุ: /html/body/div[2]/div[1]/main/main/div[2]/div[2]/div/div[2]/div[1]/div/div[4]/div[1]/table/tbody !!!
ไม่มีหน้าถัดไปสำหรับตารางที่สอง


In [ ]:
# next_page_click 1 kick

# --- ตัวอย่างการเรียกใช้กับตารางแรก ---

# กำหนด XPath ของ tbody ของตารางเป้าหมาย
main_table_xpath = "//table[contains(@class, 'table-zebra')]//tbody"

# ประมวลผลข้อมูลในหน้าแรก
print("--- กำลังประมวลผลข้อมูลตารางหลักหน้าแรก ---")
# ... (โค้ดอ่านข้อมูลของคุณ) ...

# เรียกใช้ฟังก์ชันเพื่อไปหน้าถัดไป
if go_to_next_page(driver, wait, main_table_xpath):
    print("ตอนนี้คุณอยู่ที่หน้า 2 ของตารางหลักแล้ว")
    # คุณสามารถรันโค้ดอ่านข้อมูลซ้ำอีกครั้งสำหรับหน้า 2 ได้
else:
    print("ไม่มีหน้าถัดไปสำหรับตารางหลัก")



# # กำหนดเวลารอที่ยืดหยุ่น
# long_wait = WebDriverWait(driver, 2)

# # print(f"\n--- กำลังประมวลผลหน้าปัจจุบัน... ---")

# try:
#     # 1. รอให้ตารางของหน้าปัจจุบันพร้อม
#     current_table = wait.until(EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'table-zebra')]//tbody")))
    
#     # --- (ส่วนประมวลผลข้อมูลในตาราง... ที่นี่คุณจะใส่โค้ดอ่านข้อมูลแต่ละแถว) ---
#     rows_on_this_page = current_table.find_elements(By.XPATH, ".//tr")
#     # print(f"พบ {len(rows_on_this_page)} รายการในหน้านี้.")
#     # (ย่อไว้เพื่อความกระชับ)
#     for i in range(len(rows_on_this_page)):
#         # ... process data in the current page's table ...
#         pass # Placeholder

#     # 2. ตรวจสอบว่ามีหน้าถัดไปหรือไม่
#     try:
#         # ดึงข้อความ "แสดงข้อมูล X - Y จากทั้งหมด Z"
#         pagination_text_element = driver.find_element(By.XPATH, "//div[contains(text(), 'แสดงข้อมูล')]")
#         pagination_text = pagination_text_element.text
#         numbers = re.findall(r'\d+', pagination_text)
        
#         can_go_next = False
#         if len(numbers) >= 3:
#             last_item_shown = int(numbers[1])
#             total_items = int(numbers[2])
#             if last_item_shown < total_items:
#                 can_go_next = True
        
#         # 3. ตัดสินใจคลิก "Next" ถ้าทำได้
#         if can_go_next:
#             # print(" -> พบว่ามีหน้าถัดไป, กำลังคลิก 'Next'...")
            
#             # หาและคลิกปุ่ม Next
#             next_page_button_selector = "span.mx-4 + button"
#             next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, next_page_button_selector)))
            
#             driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
#             time.sleep(0.5)
#             ActionChains(driver).move_to_element(next_button).click().perform()
            
#             # รอให้แน่ใจว่าหน้าเปลี่ยนแล้ว (รอแค่ table ใหม่ปรากฏก็พอ)
#             # print(" -> กำลังรอหน้าถัดไปโหลด...")
#             wait.until(EC.staleness_of(current_table)) # รอให้ตารางเก่าหายไป
#             wait.until(EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'table-zebra')]//tbody"))) # รอให้ตารางใหม่มา
            
#             # print(" -> เปลี่ยนหน้าสำเร็จ! Cell นี้ทำงานเสร็จแล้ว กดรันอีกครั้งเพื่อไปหน้าต่อไป")

#         else:
#             print(" -> เป็นหน้าสุดท้ายแล้ว, ไม่มีหน้าถัดไปให้กด")

#     except NoSuchElementException:
#         # ไม่เจอข้อความ Pagination เลย แสดงว่าเป็นหน้าสุดท้ายแน่นอน
#         print(" -> ไม่พบข้อความ Pagination, ไม่มีหน้าถัดไปให้กด")

# except Exception as e:
#     print(f"!!! เกิดข้อผิดพลาดร้ายแรง: {e} !!!")

--- กำลังประมวลผลข้อมูลตารางหลักหน้าแรก ---

--- กำลังตรวจสอบ Pagination ---
ตอนนี้คุณอยู่ที่หน้า 2 ของตารางหลักแล้ว


### test วนเข้าแต่ละหน่วยงาน

In [81]:
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException, ElementClickInterceptedException

organization_names = [
    "โครงการส่งน้ำและบำรุงรักษาแม่ลาว",
    "โครงการส่งน้ำและบำรุงรักษาแม่วัง"
]

TARGET_TABLE_URL = "https://cenproject.rid.go.th/track/budget?BudgetYear=2025&BudgetSourceID=1"
EXCEL_LOOKUP_FILE = 'table_main_2568.xlsx'

# --- 2. เริ่มการวนลูปหลัก ---
for org_name in organization_names:
    print(f"\n========== เริ่มต้นหน่วยงาน: {org_name} ==========")
    
    try:
        # Step 1: ค้นหาหน่วยงานในหน้าหลักแล้วคลิก
        print(f"กำลังค้นหาแถวของ: '{org_name}'...")
        project_row_xpath = f"//tbody/tr[td[2][normalize-space()='{org_name}']]"
        # รอแค่ให้แถวปรากฏก็พอ
        wait.until(EC.presence_of_element_located((By.XPATH, project_row_xpath)))
        
        # --- [แก้ไข] เปลี่ยนมาใช้ JavaScript Click เพื่อความแน่นอน ---
        # 1. หาปุ่มก่อน
        action_button_xpath = f"{project_row_xpath}//div[@data-tip='เรียกดูข้อมูลแถบเดิม']/button"
        action_button = wait.until(EC.presence_of_element_located((By.XPATH, action_button_xpath)))
        
        # 2. ใช้ JavaScript เพื่อคลิก (ป้องกัน ElementClickInterceptedException)
        print("พบปุ่ม Action, กำลังใช้ JavaScript เพื่อคลิก...")
        driver.execute_script("arguments[0].click();", action_button)
        # --- จบส่วนแก้ไข ---
        
        # Step 2: รอให้หน้ารายละเอียด (ตารางงานย่อย) โหลด
        wait.until(EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'table-zebra')]//tbody")))
        print("เข้าสู่หน้ารายละเอียดงานย่อยสำเร็จ.")

        # 3.1 หาจำนวนแถวทั้งหมดก่อน
        try:
            subtask_rows_count = len(driver.find_elements(By.XPATH, "//table[contains(@class, 'table-zebra')]//tbody/tr"))
            print(f"พบ {subtask_rows_count} งานย่อยในหน่วยงานนี้. กำลังตรวจสอบเงื่อนไข...")
        except:
            subtask_rows_count = 0
            print("ไม่พบแถวงานย่อย.")

        # Step 4: กลับไปที่หน้าหลัก
        time.sleep(10)
        print("ประมวลผลหน่วยงานนี้เสร็จสิ้น, กำลังกลับไปหน้าหลัก...")
        driver.get(TARGET_TABLE_URL)
        time.sleep(3)  # รอให้หน้าโหลดใหม่
        # wait.until(EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'table-zebra')]")))
        
    except (TimeoutException, NoSuchElementException, ElementClickInterceptedException) as e:
        # เพิ่ม ElementClickInterceptedException เข้าไปใน except block ด้วย
        print(f"!!! เกิดข้อผิดพลาดสำหรับหน่วยงาน '{org_name}': {type(e).__name__} - {str(e).splitlines()[0]}")
        print("กำลังพยายามกลับไปที่หน้าหลักเพื่อทำรายการถัดไป...")
        driver.get(TARGET_TABLE_URL)
        try:
            wait.until(EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'table-zebra')]")))
        except:
            print("!!! ไม่สามารถกลับไปยังหน้าหลักได้. หยุดการทำงาน.")
            break

    


========== เริ่มต้นหน่วยงาน: โครงการส่งน้ำและบำรุงรักษาแม่ลาว ==========
กำลังค้นหาแถวของ: 'โครงการส่งน้ำและบำรุงรักษาแม่ลาว'...


KeyboardInterrupt: 